In [0]:
%pip install nltk

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "client_id",
"fs.azure.account.oauth2.client.secret": 'secret_id',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/token_id/oauth2/token"}

In [0]:
# dbutils.fs.mount(
#     source="abfss://bing-data@bingdatastorage.dfs.core.windows.net", #container@storage_account_name
#     mount_point='/mnt/bing',
#     extra_configs =configs
# )

In [0]:

df = spark.read.load("/mnt/bing/synapse/workspaces/bing-synapse-analytics/warehouse/bing_cleaned")
display(df)

title,description,category,url,image,provider,datePublished
Georgia school shooting latest: 14-year-old suspect set to appear in court,The 14-year-old boy accused of fatally shooting four people at a Georgia high school was expected to make his first court appearance Friday.,null,https://6abc.com/post/georgia-high-school-shooting-colt-gray-teen-suspect-accused-apalachee-hs-due-court/15273486/,null,6abc News,06-09-2024
"Georgia school shooting latest: 14-year-old suspect charged with murder, father also charged",The 14-year-old suspect behind Wednesday's deadly school shooting in Georgia has been charged with four counts of felony murder.,null,https://abc13.com/post/georgia-high-school-shooting-questions-surround-weapon-motive-deadly-incident-apalachee/15270304/,null,abc13,06-09-2024
The latest in the 2024 presidential race,"With just days until the debate, Donald Trump is traveling to battleground North Carolina this afternoon while Vice President Kamala Harris is expected to continue to prepare for the event in Pennsylvania.",Politics,https://edition.cnn.com/politics/live-news/trump-harris-election-09-06-24/index.html,https://www.bing.com/th?id=OVFT.CZzav2isTPHdHcJhBXw-zi&pid=News,CNN,06-09-2024
September 6 Georgia school shooting news,The teen suspected in the deadly mass shooting at a Georgia high school and his father had initial hearings in court Friday morning. Follow for live news updates.,null,https://edition.cnn.com/us/live-news/georgia-shooting-apalachee-high-school-09-06-24,https://www.bing.com/th?id=OVFT.MjvEGDEO5zR9VQAQCFeqCS&pid=News,CNN,06-09-2024
Opinion: Canada must model Norway's balanced energy sector,"Canada needs to recognize itself as the energy giant that it is, and more importantly, realize how much greater of a giant it can be, while also being a leader in the energy transition. Unfortunately, we lack a cohesive, pragmatic culture and a fact-based narrative around energy, especially around fossil fuels, to take us to the next level.",Business,https://nationalpost.com/opinion/opinion-canada-must-model-norways-balanced-energy-sector,https://www.bing.com/th?id=OVFT.y1p7S37SRz3PBssKqc8xqi&pid=News,National Post,06-09-2024
Stunning News of Trump Sentencing Delay Sends Message: MAGA Rage Works,"The politics of this may play out fine for Kamala Harris. But once again, the legal system was cowed by Trump’s bullying and lawlessness.",Politics,https://newrepublic.com/article/185707/trump-sentencing-delay-maga-rage-works,https://www.bing.com/th?id=OVFT.E3WY-EDc1BphX5PFX_uIWS&pid=News,New Republic,07-09-2024
"Afghanistan vs New Zealand One-Off Test: Date, Time, Venue, Live streaming, Tickets & More Details",Afghanistan are set to host New Zealand for a one-off Test match in India. Here's all the crucial details about the one-off Test.,Sports,https://news.abplive.com/sports/cricket/afghanistan-vs-new-zealand-test-greater-noida-date-time-venue-live-streaming-how-to-buy-tickets-1715603,https://www.bing.com/th?id=OVFT.oxtPfWa6dhthZliiqfsSSi&pid=News,abp LIVE,07-09-2024
'Life here is much more easy': Migrants tell Sky News why PM's pledge to 'smash the gangs' won't work,"Migrants say they don't believe government efforts to tackle smuggling gangs will stop illegal migration, with ""thousands"" of people willing to take the money of those determined to get to the UK.",LifeStyle,https://news.sky.com/story/starmers-pledge-to-smash-the-gangs-wont-stop-small-boat-crossings-migrants-explain-why-13210689,https://www.bing.com/th?id=OVFT.stynnzLBt8KhhA6iVz1ATC&pid=News,Sky,06-09-2024
"Three out of four parents of children with special educational needs and disabilities forced to give up work or cut hours, Sky News learns","In a survey commissioned by Sky News, parents of children with special educational needs and disabilities (SEND) said maintaining a job is difficult due to a lack of local authority help, while others blamed not enough school support and lack of employer flexibility.",LifeStyle,https://news.sky.com/story/thre

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download VADER lexicon for sentiment analysis (run only once)
nltk.download('vader_lexicon')

# Initialize VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Define a Python function to classify sentiment
def sentiment_analysis(text):
    if text:
        sentiment_score = sid.polarity_scores(text)['compound']
        if sentiment_score >= 0.05:
            return 'positive'
        elif sentiment_score <= -0.05:
            return 'negative'
        else:
            return 'neutral'
    else:
        return 'neutral'  # You can choose to return 'neutral' for empty or null values

# Register this function as a UDF (User Defined Function)
sentiment_udf = udf(sentiment_analysis, StringType())

# Apply the UDF to add a new column to the DataFrame with sentiment category
df_with_sentiment = df.withColumn("sentiment", sentiment_udf(df["description"]))

# Show results
df_with_sentiment.select("description", "sentiment").show(truncate=False)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+
|description                                                                                                                                                                                                                                                                                                                                           |sentiment|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
# import synapse.ml.core
# from synapse.ml.services import AnalyzeText

# model=(AnalyzeText()
#             .setTextCol('description')
#             .setKing("sentimentAnalysis")
#             .setOutputCol("response")
#             .setErrorCol("error")
#             )

# result=model.transform(df)

# sentiment_df=df.withColumn("sentiment",col('response.documents.sentiment'))

# sentiment_df_final=sentiment_df.drop('error','response')


In [0]:
df_with_sentiment.display()

title,description,category,url,image,provider,datePublished,sentiment
Georgia school shooting latest: 14-year-old suspect set to appear in court,The 14-year-old boy accused of fatally shooting four people at a Georgia high school was expected to make his first court appearance Friday.,null,https://6abc.com/post/georgia-high-school-shooting-colt-gray-teen-suspect-accused-apalachee-hs-due-court/15273486/,null,6abc News,06-09-2024,negative
"Georgia school shooting latest: 14-year-old suspect charged with murder, father also charged",The 14-year-old suspect behind Wednesday's deadly school shooting in Georgia has been charged with four counts of felony murder.,null,https://abc13.com/post/georgia-high-school-shooting-questions-surround-weapon-motive-deadly-incident-apalachee/15270304/,null,abc13,06-09-2024,negative
The latest in the 2024 presidential race,"With just days until the debate, Donald Trump is traveling to battleground North Carolina this afternoon while Vice President Kamala Harris is expected to continue to prepare for the event in Pennsylvania.",Politics,https://edition.cnn.com/politics/live-news/trump-harris-election-09-06-24/index.html,https://www.bing.com/th?id=OVFT.CZzav2isTPHdHcJhBXw-zi&pid=News,CNN,06-09-2024,negative
September 6 Georgia school shooting news,The teen suspected in the deadly mass shooting at a Georgia high school and his father had initial hearings in court Friday morning. Follow for live news updates.,null,https://edition.cnn.com/us/live-news/georgia-shooting-apalachee-high-school-09-06-24,https://www.bing.com/th?id=OVFT.MjvEGDEO5zR9VQAQCFeqCS&pid=News,CNN,06-09-2024,negative
Opinion: Canada must model Norway's balanced energy sector,"Canada needs to recognize itself as the energy giant that it is, and more importantly, realize how much greater of a giant it can be, while also being a leader in the energy transition. Unfortunately, we lack a cohesive, pragmatic culture and a fact-based narrative around energy, especially around fossil fuels, to take us to the next level.",Business,https://nationalpost.com/opinion/opinion-canada-must-model-norways-balanced-energy-sector,https://www.bing.com/th?id=OVFT.y1p7S37SRz3PBssKqc8xqi&pid=News,National Post,06-09-2024,positive
Stunning News of Trump Sentencing Delay Sends Message: MAGA Rage Works,"The politics of this may play out fine for Kamala Harris. But once again, the legal system was cowed by Trump’s bullying and lawlessness.",Politics,https://newrepublic.com/article/185707/trump-sentencing-delay-maga-rage-works,https://www.bing.com/th?id=OVFT.E3WY-EDc1BphX5PFX_uIWS&pid=News,New Republic,07-09-2024,negative
"Afghanistan vs New Zealand One-Off Test: Date, Time, Venue, Live streaming, Tickets & More Details",Afghanistan are set to host New Zealand for a one-off Test match in India. Here's all the crucial details about the one-off Test.,Sports,https://news.abplive.com/sports/cricket/afghanistan-vs-new-zealand-test-greater-noida-date-time-venue-live-streaming-how-to-buy-tickets-1715603,https://www.bing.com/th?id=OVFT.oxtPfWa6dhthZliiqfsSSi&pid=News,abp LIVE,07-09-2024,neutral
'Life here is much more easy': Migrants tell Sky News why PM's pledge to 'smash the gangs' won't work,"Migrants say they don't believe government efforts to tackle smuggling gangs will stop illegal migration, with ""thousands"" of people willing to take the money of those determined to get to the UK.",LifeStyle,https://news.sky.com/story/starmers-pledge-to-smash-the-gangs-wont-stop-small-boat-crossings-migrants-explain-why-13210689,https://www.bing.com/th?id=OVFT.stynnzLBt8KhhA6iVz1ATC&pid=News,Sky,06-09-2024,negative
"Three out of four parents of children with special educational needs and disabilities forced to give up work or cut hours, Sky News learns","In a survey commissioned by Sky News, parents of children with special educational needs and disabilities (SEND) said maintaining a job is difficult due to a lack of local authority help, while others blamed not enough school sup

In [0]:
from pyspark.sql.utils import AnalysisException

try:
    table_name='default.sentiment_bing'
    df_with_sentiment.write.format('delta').saveAsTable(table_name)
except AnalysisException:
    print("table is already created")

    df_with_sentiment.createOrReplaceTempView('temp')

    spark.sql(f"""merge into {table_name} target_table
                    using temp source_view
                    on  source_view.url=target_table.url

                    when matched and
                    source_view.title<>target_table.title OR
                    source_view.description<>target_table.description OR
                    source_view.provider<>target_table.provider OR
                    source_view.image<>target_table.image OR
                    source_view.category<>target_table.category OR
                    source_view.datePublished<>target_table.datePublished OR
                    source_view.sentiment<>target_table.sentiment
                    Then update set *

                    when not matched then insert *
    """)